In [4]:
import cv2 as cv
import numpy as np
import sklearn as skl
import scipy as cp
import matplotlib.pyplot as plt
import tensorflow as tf
import torch as pt
import os
import csv
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import manifold, neighbors, metrics
from scipy import signal



In [5]:

#pathTest = currentLocation + '/data/sign_mnist_test.csv'
#test = pd.read_csv(pathTest)
#labelsTest = test['label'].values
#unique_valTest = np.array(labelsTest)
#np.unique(unique_valTest)
#label_binrizerTest = LabelBinarizer()
#labels = label_binrizer.fit_transform(labelsTest)
#train.drop('label', axis = 1, inplace = True)
#imagesTest = trainTest.values

In [6]:
### Dictionaries for labeling data from images
def retrieveDict(nr):
    if nr == 0:
        dictionary = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26}
    elif nr == 1:
        dictionary = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}
    else:
        print("No valid dictionary found")
        dictionary = -1
    
    return dictionary

In [7]:
### Data reading functions

def readDataFolder(path, dictionary, oneHot,rescaleSize):
    '''read data from different folders
        data location: .../path/
        labels assigned from dictionary default A = 1, B = 2 etc
        oneHot determines if labels are made binary i.e. A =[1,0,...], B=[0,1,...] etc
        returns data, rescaled to rescaleSize,rescaleSize and labels as numpy arrays
        '''
    data = []
    labels = []
    size = rescaleSize,rescaleSize
    folder = 'A'
    #for folder in os.listdir(path):
    #print(folder, end = ' | ')
    for image in os.listdir(path + "/" + folder):
        img = cv.imread(path + '/' + folder + '/' + image)
        img = cv.resize(img,size)
        data.append(img)
        labels.append(dictionary[folder])
    data = np.array(data)
    data = data.astype('float32')/255.0
    if oneHot == 1:
        labels = tf.keras.utils.to_categorical(labels)
    else:
        labels = np.array(labels)
    
    return data, labels

def readCSV(path, oneHot, rescaleSize):
    '''read data from csv file
        data location: .../path/
        oneHot determines if labels are made binary i.e. A =[1,0,...], B=[0,1,...] etc or not
        returns data, rescaled to rescaleSize,rescaleSize, and labels as numpy arrays
        '''
    size = rescaleSize,rescaleSize
    data = []
    train = pd.read_csv(path)
    labels = train['label'].values
    unique_val = np.array(labels)
    np.unique(unique_val)
    label_binrizer = LabelBinarizer()
    labels = label_binrizer.fit_transform(labels)
    train.drop('label', axis = 1, inplace = True)
    images = train.values
    if oneHot == 1:
        labels = tf.keras.utils.to_categorical(labels)
        
    for i in range(len(images)):
        img = images[i].reshape(28,28)
        img = cv.resize(img,size)
        data.append(img)
    data = np.array(data)
    data = data.astype('float32')/255.0
    
    return data, labels

In [46]:
def preprocessing(method, data, size):
    '''
        Applies preprocessing to data
            
        '''
    # Parameters
    gaussianKernel = 5
    gaussianMean = 0
    structElement = np.ones((3,3),np.uint8)
    cannyKernel = 3
    lowThreshold = 40
    highThreshold = 120
    
    if method[0] == 1:
        print("Not implemented, Doing Gamma Correction")
                       
    if method[1] == 1:
        print("Gaussian Denoising")
        data = gaussianDenoise(data, gaussianKernel, gaussianMean, size)
        
    if method[2] == 1:
        print("Converting to grayscale")
        data = grayScale(data)
        
    if method[3] == 1:
        print("Canny Edge detection")
        data = cannyEdgeDetection(data,lowThreshold,highThreshold,cannyKernel)
        
    if method[4] == 1:
        print("Segmentation, background/foreground")     
        data = segmentation(data,structElement)
        
    if method[5] == 1:
        print("Sobel Filtering")
        data = sobelFilter(data)
        
    if method[6] == 1:
        print("Performing opening")
        data = opening(data,structElement)
    return data

def cannyEdgeDetection(inData, lowThreshold,highThreshold, kernelSize):
    data = []
    for i in range(len(inData)):
        cannyEdges = cv.Canny(inData[i].astype("uint8"), lowThreshold, highThreshold, kernelSize)
        print(np.shape(cannyEdges))
        data.append(cannyEdges)
    data = np.array(data)
    print(np.shape(data))
    return
  
def grayScale(inData):
    data = []
    for i in range(len(inData)):
        gray = cv.cvtColor(inData[i]*255, cv.COLOR_RGB2GRAY).astype("uint8") #Convert to grayscale
        data.append(gray)
    data = np.array(data)
    return data

        
def gaussianDenoise(inData, kernel, mu, size):
    data = []
    print(np.shape(inData))
    for i in range(len(inData)):
        blurred = cv.GaussianBlur(inData[i], (kernel,kernel),mu)
        data.append(blurred)
    data = np.array(data)
    return data
        
        
def adjustGamma(image, gamma = 1.0): # Try to implement to get working for images
    invGamma = 1/gamma
    table = np.array([((i/255.0)**invGamma)*255
        for i in np.arange(0,256)]).astype("uint8")
    return cv.LUT(image,table)

def segmentation(inData,structElement,grayed):
    data = []
    for i in range(len(inData)): 
        if grayed == 0: 
            gray = cv.cvtColor(inData[i]*255, cv.COLOR_RGB2GRAY).astype("uint8") #Convert to grayscale
        elif grayed == 1:
            gray = inData[i]
            
        thresholding1 = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,11,2) #perform adaptive thresholing
        opening = cv.morphologyEx(thresholding1 , cv.MORPH_OPEN, structElement, iterations=2) #perform opening
        distance_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
        data.append(distance_transform)
    data = np.array(data)
    return data

def opening(inData,structElement):
    data = []
    for i in range(len(inData)):
        opening = cv.morphologyEx(inData[i] , cv.MORPH_OPEN, structElement, iterations=2) #perform opening
        data.append(opening)
    data = np.array(data)
    return data

def sobelFilter(inData):
    data = []
    for i in range(len(inData)):
        sobel = cv.Sobel(inData[i],cv.CV_64F,0,1,ksize=3)
        data.append(sobel)
    data = np.array(data)
    return data

In [ ]:
### Visualize things

def plotSamples(data, samples, size):
    fig, axs = plt.subplots(1, samples, figsize=(15, 4), sharey=True)
    for i in range(samples): 
        axs[i].imshow(data[np.random.randint(len(data),size = 1)].reshape(size,size))
    

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.3, stratify = labels, random_state = 7)

def augment(origData, origLabel, runs, strength):
    """
    Augments the data by naively aplying random gaussian noise to array, applies same strength to whole array
        :param original data, original labels, how many times to augment data, the strength of the noise
        :return: augmented list of data and labels
        """
    labelToAppend = origLabel
    dataToPermutate = origData
    for i in range(runs): #For each augmentation
        origLabel = np.vstack((origLabel,labelToAppend)) #Extend labels since wwe don't want to change them at all
        origData = np.vstack((origData, plusNoise(dataToPermutate, strength))) #Add data where noise have been added
        origLabel = np.vstack((origLabel,labelToAppend)) #Extend labels again
        origData = np.vstack((origData, negNoise(dataToPermutate, strength))) #Add data where noise have been subtracted
    return origData, origLabel

def plusNoise(inData, strength):
    """
    Add gausian white noise to data
        :param Data to apply noise to, strength of noise to apply
        :return: Data where noise have been added
        """
    data = inData
    for i in range(len(inData)): # For each element
        data[i] = inData[i]+np.random.normal(0, strength, size=(len(data[0]))) #Add noise
    return data

def negNoise(inData, strength):
    """
    Subtract gaussian white noise from data
        :param Data to subtract from, strength of noise
        :return: Data where noise have been subtracted
        """
    data = inData
    for i in range(len(inData)): # for each element
        data[i] = inData[i] + np.random.normal(0, strength, size=(len(data[0]))) # Remove noise
    return data
print(np.shape(y_train))
x_train, y_train = augment(x_train,y_train,4,0.01)
print(np.shape(y_train))

In [ ]:
dt = tree.DecisionTreeClassifier()#Create model
dt.fit(x_train,y_train) #Train model

In [ ]:

prediction = dt.predict(x_test) #Do prediction on test data
accuracy = metrics.accuracy_score(y_test, prediction) #Calculate accuracy from predictions
print("\n Accuracy: %f",accuracy)
print("\n Results from classifier:  \n %s \n" %( metrics.classification_report(y_test, prediction)))
print("\n Confussion matrix:\n %s" % metrics.confusion_matrix(y_test, prediction)) #PRint confusion matrix
#cnf_matrix = confusion_matrix(testLabels, prediction) #create matrix for plotting
#plot_confusion_matrix(cnf_matrix, classes=keys, title='Normalized confusion matrix')
#kfold = 10
#scores = cross_val_score(dt, images , labels, cv=10) #Do cross validation
#print("\n Cross validation score: \n")
#print(scores) #Report scores of cross validation
#print("\n Mean cv score: %f", np.mean(scores))

In [ ]:
prediction = dt.predict(test) #Do prediction on test data
accuracy = metrics.accuracy_score(labelsTest, prediction) #Calculate accuracy from predictions
print("\n Accuracy: %f",accuracy)
print("\n Results from classifier:  \n %s \n" %( metrics.classification_report(labelsTest, prediction)))


In [ ]:

lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")

In [ ]:

#Load images
currentLocation = os.getcwd()
imgB = cv.imread(currentLocation + '/data/aBright.jpg')
imgD = cv.imread(currentLocation + '/data/aWood.jpg')
imgS =  cv.imread(currentLocation + '/data/aSilhouette.jpg')
imgTB = cv.imread(currentLocation + '/data/aTopBright.jpg')
gB = cv.cvtColor(imgB, cv.COLOR_BGR2GRAY)


edges1 = cv.Canny(gB,100,200)
blur = cv.GaussianBlur(gB,(5,5),0)
edges2 = cv.Canny(blur,100,200)
loadedImages = np.vstack((edges1,np.vstack((edges2,gB))))
cv.imshow('Stacked', loadedImages)
cv.waitKey(0)
cv.destroyAllWindows()

kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11, 11))

#Convert to HSV
hsvBright = cv.cvtColor(imgB, cv.COLOR_BGR2HSV)
skinMaskB = cv.inRange(hsvBright, lower, upper)
skinMaskB = cv.erode(skinMaskB, kernel, iterations = 2)
skinMaskB = cv.dilate(skinMaskB, kernel, iterations = 2)
skinMaskB = cv.GaussianBlur(skinMaskB, (3, 3), 0)
skinB = cv.bitwise_and(imgB, imgB, mask = skinMaskB)

hsvDark = cv.cvtColor(imgD, cv.COLOR_BGR2HSV)
skinMaskD = cv.inRange(hsvDark, lower, upper)
skinMaskD = cv.erode(skinMaskD, kernel, iterations = 2)
skinMaskD = cv.dilate(skinMaskD, kernel, iterations = 2)
skinMaskD = cv.GaussianBlur(skinMaskD, (3, 3), 0)
skinD = cv.bitwise_and(imgD, imgD, mask = skinMaskD)


hsvSilhouette = cv.cvtColor(imgS, cv.COLOR_BGR2HSV)
skinMaskS = cv.inRange(hsvSilhouette, lower, upper)
skinMaskS = cv.erode(skinMaskS, kernel, iterations = 2)
skinMaskS = cv.dilate(skinMaskS, kernel, iterations = 2)
skinMaskS = cv.GaussianBlur(skinMaskS, (3, 3), 0)
skinS = cv.bitwise_and(imgS, imgS, mask = skinMaskS)


hsvTopBright = cv.cvtColor(imgTB, cv.COLOR_BGR2HSV)
skinMaskTB = cv.inRange(hsvTopBright, lower, upper)
skinMaskTB = cv.erode(skinMaskTB, kernel, iterations = 2)
skinMaskTB = cv.dilate(skinMaskTB, kernel, iterations = 2)
skinMaskTB = cv.GaussianBlur(skinMaskTB, (3, 3), 0)
skinTB = cv.bitwise_and(imgTB, imgTB, mask = skinMaskTB)

H, S, V = cv.split(hsvDark) 

loadedImages = np.vstack((skinD,imgD))
#loadedImages = np.vstack((loadedImages,V))
cv.imshow('Stacked', loadedImages)
cv.waitKey(0)
cv.destroyAllWindows()
#

In [49]:
def main():
    '''Main method for readability of code'''
    ## Parameters
    dataset = 0 # 0 for significant ASL, 1 for ASL dataset, 2 for MNIST dataset
    binaryLabel = 1 # 0 for encoded i.e. A=1, B=2 etc 1 to one hot encode data i.e. A =[1,0,...], B = [0,1,...] etc
    rescaleSize = 64 # rescale images to size rescaleSize,rescaleSize 
    currentLocation = os.getcwd()
    vissualize = 0
    
    # preprocess methods
    gammaCorrection = 0 # Do gamma correction to move average pixel intesity to range 100-150 Not implemented yet
    gaussianBlur = 0 # If we apply gaussian blur to images
    grayScale = 0 #Convert to grayscale automatically done if using segmentation
    edgeDetection = 1 # If we do edge detection, using canny edge detector
    segmentation = 0 # Try to segment image into foreground and background 
    sobelFilter = 0 # Try to segment image into different objects
    opening = 0 # Do morphological opening, erosion followed by dilation
    
    methods = []
    methods.append(gammaCorrection)
    methods.append(gaussianBlur)
    methods.append(grayScale)
    methods.append(edgeDetection)
    methods.append(segmentation)
    methods.append(sobelFilter)
    methods.append(opening)

    
    ## Load data
    if dataset == 0:
        path = currentLocation +'/significant-asl-sign-language-alphabet-dataset/Training-Set'
    elif dataset == 1:
        path = currentLocation + '/asl-alphabet/asl_alphabet_train/asl_alphabet_train' 
    elif dataset == 2:
        path = currentLocation + '/data/sign_mnist_train.csv' 
    else:
        print('Invalid dataset')
        exit()
    
    dictionary = retrieveDict(dataset)
    print('Reading data.')
    if dataset == 0 or dataset ==  1:
        data, labels = readDataFolder(path, dictionary, binaryLabel, rescaleSize)
    elif dataset == 2:
        data, labels = readCSV(path,binaryLabel, rescaleSize)
    
    # Visualize some samples of data
    if vissualize == 1:
        print('Visualize 5 samples from data.')
        plotSamples(data,5,rescaleSize)
    # Preprocessing
    print(type(data))
    print(np.shape(data))
    cv.imshow('Stacked', data[2538])
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    #data = preprocessing(methods, data, rescaleSize)
    data = preprocessing(methods, data, rescaleSize)
    
    

    # opening
    cv.imshow('Stacked', data[2538])
    cv.waitKey(0)
    cv.destroyAllWindows()

In [50]:
main()

Reading data.
<class 'numpy.ndarray'>
(2976, 64, 64, 3)
Canny Edge detection
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 

(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(

(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(

TypeError: 'NoneType' object is not subscriptable